In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import os, sys

In [2]:
sys.path.append(os.path.abspath('../'))
import sfr_methods
from utils import load_dict

In [3]:
plt.style.use('seaborn')
plt.style.use('seaborn-colorblind')

In [4]:
final_paths = np.load('../Selecting_Sample/final_sample_paths.npy', allow_pickle=True)

In [5]:
def integrated_sfr(galdf, galdict, avg, threshold):
    df = galdf.copy()
    
    df = df.dropna()
    df = df[(df['r/re'] < 1.5) & (df['r/re'] > 0.1)]
    df = df[(df['comp'] == 0) & (df['agn'] == 0) & (df['seyfert'] == 0) & (df['liner'] == 0)]
    
    spiral_df = df[(df['sp_{}{}'.format(threshold[0], threshold[1])] == True) & ('nsp_{}{}'.format(threshold[0], threshold[1]) == False)]
    nspiral_df = df[df['nsp_{}{}'.format(threshold[0], threshold[1])] == False]
    
    spiral_sfr, sig_spiral_sfr = sfr_methods.get_sfr(spiral_df.index, galdf, galdict, avg=avg)
    nspiral_sfr, sig_nspiral_sfr = sfr_methods.get_sfr(nspiral_df.index, galdf, galdict, avg=avg)
    
    return spiral_sfr, sig_spiral_sfr, nspiral_sfr, sig_nspiral_sfr

In [6]:
def calculate_stdv(series):
    mean = series.mean()
    
    summ = 0
    for element in series:
        summ = summ + (element - mean)**2
    
    return np.sqrt(summ / len(series))

In [7]:
%%time
dict_list = []
df_list = []

for path in final_paths:
    filename = path.split('/')[-1]
    
    galdict = load_dict('../Make_Resources/Dicts/' + filename + '.pkl')
    df = pd.read_pickle('../Make_Resources/DFs/' + filename + '.pkl')
    
    dict_list.append(galdict)
    df_list.append(df)

CPU times: user 6.54 s, sys: 32.9 s, total: 39.4 s
Wall time: 44.8 s


In [8]:
thresholds = []

for i in range(1, 8):
    for j in range(1, 8):
        thresholds.append((i, j))

In [ ]:
%%time

for i in range(len(final_paths)):
    galdict = dict_list[i]
    df = df_list[i]
    
    k = 0
    for threshold in thresholds:
        spiral_sfr, sig_spiral_sfr, nspiral_sfr, sig_nspiral_sfr = integrated_sfr(df, galdict, True, threshold)
        
        galdict['avg_sp_sfr_{}{}'.format(threshold[0], threshold[1])] = spiral_sfr
        galdict['sig_avg_sp_sfr_{}{}'.format(threshold[0], threshold[1])] = sig_spiral_sfr
        
        galdict['avg_nsp_sfr_{}{}'.format(threshold[0], threshold[1])] = nspiral_sfr
        galdict['sig_avg_nsp_sfr_{}{}'.format(threshold[0], threshold[1])] = sig_nspiral_sfr
        
        print(k)
        k += 1
        
    dict_list[i] = galdict
    print('Galaxy #', i)

Returning ha emission = 0 as no spaxels in bin. MaNGA ID: 1-178542
Returning hb emission = 0 as no spaxels in bin. MaNGA ID: 1-178542
Returning SFR = 0 as bin results in Ha or Hb == 0. MaNGA ID: 1-178542
H-a covariance file does not exist. Calculating...
H-b covariance file does not exist. Calculating...
0
Returning ha emission = 0 as no spaxels in bin. MaNGA ID: 1-178542
Returning hb emission = 0 as no spaxels in bin. MaNGA ID: 1-178542
Returning SFR = 0 as bin results in Ha or Hb == 0. MaNGA ID: 1-178542
H-a covariance file does not exist. Calculating...
H-b covariance file does not exist. Calculating...
1
Returning ha emission = 0 as no spaxels in bin. MaNGA ID: 1-178542
Returning hb emission = 0 as no spaxels in bin. MaNGA ID: 1-178542
Returning SFR = 0 as bin results in Ha or Hb == 0. MaNGA ID: 1-178542
H-a covariance file does not exist. Calculating...
H-b covariance file does not exist. Calculating...
2
Returning ha emission = 0 as no spaxels in bin. MaNGA ID: 1-178542
Returning

In [ ]:
def save_dict(galdict, name):
    with open(name + '.dict.pkl', 'wb') as f:
        pickle.dump(galdict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
for galdict in dict_list:
    save_dict(galdict, '../Form_Resources/Dicts_Avg_SFR_Thresholds/' + galdict['filename'].split('.')[0])